In [ ]:
# 고유명사 어쩌지? 

### 자모 테스트

In [1]:
from gensim.models import FastText

# 자모 변환된 한국어 문서를 불러오기
with open("corpus.txt", "r", encoding="utf-8") as f:
    sentences = [line.strip().split() for line in f.readlines()] 

# FastText 모델 학습
model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4, sg=1, epochs=10)

# 학습한 모델 저장
model.save("fasttext_jamo.model")
print("FastText 자모 단위 모델 학습 완료!")


FastText 자모 단위 모델 학습 완료!


In [ ]:
from gensim.models import FastText

# 학습된 모델 불러오기
model = FastText.load("fasttext_jamo.model")

# 테스트 단어 (자모 분리)
test_word = decompose_korean("자연어")
print(f"자모 변환된 단어: {test_word}")

# 단어 벡터 출력
print(f"'{test_word}'의 벡터 값:\n", model.wv[test_word])

In [ ]:
# 두 문장의 코사인 유사도 계산
def calculate_similarity(sentence1, sentence2, model):
    vec1 = sentence_to_vector(sentence1, model)
    vec2 = sentence_to_vector(sentence2, model)
    return cosine_similarity([vec1], [vec2])[0][0]  # 코사인 유사도 계산

### JSON TEST

In [19]:
import json

# JSON 데이터 불러오기
with open('json_data.json', 'r', encoding='utf-8') as f:
    artwork_data = json.load(f)

# 데이터 샘플 출력
print(artwork_data[0])  # 첫 번째 작품 정보 출력


{'title': '작가사진', 'title_ch': '作家寫眞', 'title_eng': 'Selfportrait', 'artist': '한기석', 'artist_eng': 'HAN Kisuk', 'artwork_number': 1, 'year': '1960', 'size': '41×51', 'materials': '종이에 젤라틴실버프린트', 'category': '사진', 'description': '‘농(Nong)’이라는 이름으로 미국에서 널리 알려진 한농(韓農) 한기석(1930-2011)은 국내 활동이 그리 많지 않아서 한국 화단에서는 생소한 이름이다. 그가 최초로 한국 화단에 등장한 것은 1971년 11월 신세계 화랑에서 개최한《Nong 展》이후이다. 그는 농(Nong)을 구름 위의 시선(詩仙) 혹은 주선(酒仙)같은 존재로 비유해서 미국에서 자신의 이름으로 쓰고 있다.그의 작품은 전반적으로 자신의 철학적 이미지를 조형화시킨 추상 회화 계통이다. 일종의 형이상학적인 회화 혹은 초현실적인 환상세계라고도 할 수 있는 그의 작품은 양식적인 면에서 주로 구상적인 형태를 취한다.한기석의 <작가사진>(1960)은 본인의 얼굴을 찍은 것으로, 사진 속에서 작가는 자신의 작품을 배경으로 화면의 우측을 주시하고 있다.', 'read_count': 10468}


FastText 자모 모델 학습 완료!


In [27]:
import numpy as np

# FastText 기반 유사도 계산 함수
def cosine_similarity(vec1, vec2):
    """코사인 유사도 계산"""
    return np.dot(vec1, vec2.T) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def get_embedding(sentence, model):
    """문장 임베딩 계산 (FastText)"""
    tokens = sentence.split()  # 문장을 공백 기준으로 나눔
    vec = np.mean([model.wv[token] for token in tokens if token in model.wv], axis=0)  # 평균 벡터 계산
    return vec

# RAG 정답과 LLM 응답의 유사도를 계산하는 함수
def calculate_similarity(ground_truth, llm_response, model):
    # RAG 정답과 LLM 응답에 대한 임베딩 계산
    vec1 = get_embedding(ground_truth, model)
    vec2 = get_embedding(llm_response, model)
    
    # 유사도 계산
    similarity = cosine_similarity(vec1, vec2)
    return similarity


In [33]:
# 예시 문장들 (ground_truth와 LLM 응답)
rag_answer = """
집에가고싶어.
"""


llm_response = """
짜증이나.
"""

# 유사도 계산
similarity = calculate_similarity(rag_answer, llm_response, model)

print(f"RAG 정답과 LLM 응답의 유사도: {similarity:.4f}")



RAG 정답과 LLM 응답의 유사도: 0.1882


In [ ]:
# 잠시만 질문도 뭔지 파악해야되는거아냐? 정답이랑 llm답변만 비교하는게 아니라

### 커스텀

In [101]:
import re
import numpy as np
from gensim.models import FastText
from kiwipiepy import Kiwi

# ✅ FastText 모델 로드
model_path = "./fasttext_jamo_with_numbers.model"
model = FastText.load(model_path)
kiwi = Kiwi()

# ✅ 문장을 FastText 벡터로 변환하는 함수
def get_sentence_embedding(sentence, model):
    tokens = [token.form for token in kiwi.tokenize(sentence)]
    vectors = [model.wv[word] for word in tokens if word in model.wv]

    if len(vectors) == 0:
        return np.zeros(model.vector_size)  # 모델에 없는 단어만 있으면 0 벡터 반환
    return np.mean(vectors, axis=0)  # 평균 벡터 반환

# ✅ 숫자 비교 및 패널티 적용 함수
def compare_numbers_ignore_order(sentence1, sentence2):
    """숫자의 값이 다르면 패널티 적용, 순서는 무시"""
    numbers1 = set(re.findall(r'\d+', sentence1))  # 숫자만 추출하여 집합으로 저장
    numbers2 = set(re.findall(r'\d+', sentence2))

    if not numbers1 or not numbers2:
        return 1.0  # 숫자가 없으면 패널티 없음

    if numbers1 != numbers2:  
        return 0.1  # 숫자 값이 다르면 패널티 적용

    return 1.0  # 숫자 값이 같으면 패널티 없음



In [107]:
# ✅ 무시할 단어 목록 (조사, 접속사 등)
IGNORED_WORDS = {"이", "그", "그리고", "하지만", "또한", "그러나", "즉"}

# ✅ 중요한 개념을 담고 있는 키워드 목록
KEYWORDS = {"작품", "제작", "연도", "년도", "미술", "예술", "작가", "전시", "소재"}

# ✅ 할루시네이션 감지 함수 개선
def detect_hallucination(ground_truth, generated_text):
    """핵심 개념이 유지되면 감점하지 않고, 새로운 정보 추가 또는 정보 삭제 시 감점"""
    
    # ✅ 형태소 분석 후 명사, 동사, 형용사만 비교
    gt_tokens = {token.form for token in kiwi.tokenize(ground_truth) if token.tag.startswith(("N", "V", "X"))}
    gen_tokens = {token.form for token in kiwi.tokenize(generated_text) if token.tag.startswith(("N", "V", "X"))}

    # ✅ 불필요한 단어(조사, 접속사 등) 제거
    gt_tokens -= IGNORED_WORDS
    gen_tokens -= IGNORED_WORDS

    # ✅ 원래 있어야 할 단어 중 중요한 개념 단어만 체크
    gt_key_tokens = gt_tokens & KEYWORDS
    gen_key_tokens = gen_tokens & KEYWORDS

    # ✅ 핵심 개념이 유지되었다면 감점하지 않음
    if gt_key_tokens and gen_key_tokens and gt_key_tokens == gen_key_tokens:
        return 1.0  # 핵심 개념이 유지되었으므로 감점 X

    # ✅ 새로운 개념이 2개 이상 추가되면 감점
    extra_tokens = gen_tokens - gt_tokens  # 생성된 문장에서 원문에 없는 단어 찾기
    missing_tokens = gt_tokens - gen_tokens  # 원래 문장에서 빠진 단어 찾기

    if len(extra_tokens) > 2:  # 새로운 정보가 추가되었을 때 감점
        return 0.5

    # ✅ 중요한 정보가 빠지면 감점
    if len(missing_tokens) > 2:
        return 0.5

    return 1.0  # 큰 차이가 없으면 점수 유지


def custom_similarity(text1, text2, model):
    """FastText 기반 유사도 계산, 동의어 적용, 숫자 패널티 반영, 할루시네이션 감지"""
    # 동의어 확장 적용
    # 문장 임베딩 변환 (FastText)
    vec1 = get_sentence_embedding(text1, model)
    vec2 = get_sentence_embedding(text2, model)

    if np.linalg.norm(vec1) == 0 or np.linalg.norm(vec2) == 0:
        return 0.0  # 한쪽 문장이라도 FastText 벡터가 없으면 유사도 0

    # 코사인 유사도 계산
    similarity = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

    # 숫자 비교 후 패널티 적용 (숫자 순서 무시)
    number_penalty = compare_numbers_ignore_order(text1, text2)

    # 할루시네이션 감지 후 패널티 적용
    hallucination_penalty = detect_hallucination(text1, text2)

    # 최종 유사도 계산 (숫자 & 할루시네이션 패널티 반영)
    final_score = similarity.item() * number_penalty * hallucination_penalty  # numpy 값 -> float 변환

    return final_score


In [108]:
# ✅ 예제 문장
text1 = "이 작품은 1998년에 제작되었습니다."
text2 = "이 작품은 1998년에 만들어졌습니다."
text3 = "이 작품의 제작 연도는 1998년입니다."
text4 = "이 작품은 1999년에 제작되었습니다."
text5 = "이 작품은 1998년이 아닌 2000년에 제작되었습니다."

# ✅ 숫자 + 문맥 + 동의어까지 고려한 유사도 비교
sim1 = custom_similarity(text1, text2, model)
sim2 = custom_similarity(text1, text3, model)
sim3 = custom_similarity(text1, text4, model)
sim4 = custom_similarity(text1, text5, model)

print(f"🔹 유사도 (동일 의미 다른 표현): {sim1:.4f}")  # 높은 점수 기대
print(f"🔹 유사도 (의미 유지, 단어 순서 다름): {sim2:.4f}")  # 높은 점수 기대
print(f"🔹 유사도 (숫자 다름): {sim3:.4f}")  # 낮은 점수 (패널티 적용)
print(f"🔹 유사도 (숫자 다름, 추가 숫자 있음): {sim4:.4f}")  # 매우 낮은 점수


🔹 유사도 (동일 의미 다른 표현): 0.9999
🔹 유사도 (의미 유지, 단어 순서 다름): 0.9495
🔹 유사도 (숫자 다름): 0.0864
🔹 유사도 (숫자 다름, 추가 숫자 있음): 0.0921


In [113]:
# RAG 정답과 LLM 응답 예제
rag_answer = """
김세진이 컴퓨터 그래픽 회사에서 익힌 다양한 최신 영상 기법을 활용하여 '되돌려진 시간'에서 실험성과 당대성을 확인할 수 있는 감각적인 비디오 작품을 만들었습니다. 

이 작품은 각각의 화면에는 울기, 성냥 켜기, 그리기, 말하기, 머리 말리기, 먹기 등 일상적인 행위를 촬영한 후에 리버스(reverse) 기법으로 되돌린 영상이 재생되며, 

평범한 순간을 담은 영상을 되돌렸을 때에 발생하는 낯선 공감각적 내러티브를 통해 또 다른 차원의 이미지를 제시하고자 한 작가의 시도를 보여줍니다.

"""


llm_response = """
김세진은 컴퓨터 그래픽 회사에서 다양한 최신 영상 기법을 익혀 "되돌려진 시간"에서 이러한 기술을 활용하여 6채널 영상을 제작했습니다. 

이를 통해 시간의 물리적 흐름을 리버스하는 복잡한 시각적 표현을 구현하고, 매체 실험성을 강화했습니다

"""

# 유사도 계산
similarity = custom_similarity(rag_answer, llm_response, model)

print(f"RAG 정답과 LLM 응답의 유사도: {similarity:.4f}")




RAG 정답과 LLM 응답의 유사도: 0.4536


In [ ]:
# 점수범위 -1 ~ 1 
# 1: 완전 같음
# 0: 완전 무관
# -1: 반대되는 의미이지만 거의 발생 x

# 0.9612, 0.7238, 0.9073, 0.9053, 0.9434, 0.4258

# 숫자가 있을때는 custom similartiy를 아니라면 calculate_similarity를 사용 